In [ ]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta
import os, psutil
import holidays
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import LSTM, Dense
start_time = time.time()
# Función para solicitar al usuario una fecha y convertirla en un objeto datetime
def solicitar_fecha(mensaje):
    while True:
        fecha_str = input(mensaje)
        try:
            return datetime.strptime(fecha_str, '%Y-%m-%d')
        except ValueError:
            print("Error: El formato de la fecha no es correcto. Utilice el formato AAAA-MM-DD.")

# Solicitar al usuario que ingrese las fechas de inicio y fin
fecha_inicio = solicitar_fecha("Ingrese la fecha de inicio (formato AAAA-MM-DD): ")
fecha_fin = solicitar_fecha("Ingrese la fecha de fin (formato AAAA-MM-DD): ")
# Inicializar el diccionario para almacenar los DataFrames

diccionario_datasets = {}

# Iterar sobre el rango de fechas

fecha_actual = fecha_inicio
while fecha_actual <= fecha_fin:
    fecha_str = fecha_actual.strftime('%Y%m%d')
    url = f"https://storage.googleapis.com/validaciones_tmsa/Salidas/salidas{fecha_str}.csv"

    response = requests.get(url)
    if response.status_code == 200:
        csv_data = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(csv_data))

        # Almacenar el DataFrame en el diccionario con la fecha como clave
        diccionario_datasets[fecha_str] = df
    else:
        print(f"No se pudo acceder al archivo CSV para la fecha {fecha_str}. Estado de respuesta: {response.status_code}")

    fecha_actual += timedelta(days=1)

# Acceder a un DataFrame específico
# Por ejemplo, para acceder al dataset del 1 de enero de 2023

#df_especifico = diccionario_datasets['20231129']
#print(df_especifico.head())

# Concatenar todos los DataFrames en el diccionario en un único DataFrame
df_concatenado = pd.concat(diccionario_datasets.values(), ignore_index=True)

process = psutil.Process()
# Print the memory usage in megabytes
print(f"Memory usage: {process.memory_info().rss / (1024 * 1024)} megabytes")

end_time = time.time()
duration = end_time - start_time

print(f"El script tardó {duration} segundos en ejecutarse.")
# Mostrar las primeras filas del DataFrame concatenado

df_concatenado.columns = ['Transaction_date','Time', 'Operator', 'Line', 'Station','Station_Access', 'Device','Inputs','Outputs','archive']

# Suponiendo que df_concatenado es tu DataFrame existente
data1 = df_concatenado
data1

# Dropping the 'Operador' and 'Linea' columns
data1_dropped = data1.drop(columns=['Operator', 'Line', 'archive'])
filtered_data0 = data1_dropped[(data1_dropped['Inputs'] == 0) & (data1_dropped['Outputs'] == 0)]
filtered_data0
filtered_data = data1_dropped[(data1_dropped['Inputs'] != 0) | (data1_dropped['Outputs'] != 0)]
filtered_data

#Filtrar las filas que tienen un valor en la columna 'Tiempo' mayor a '03:45:00'
filtered_time_data = filtered_data[filtered_data['Time'] >= '03:45:00']
# Mostrar las primeras filas del DataFrame filtrado por tiempo
unique = len(filtered_time_data['Station'].unique())
unique

#filtered_time_data.to_csv('salidas20230809EngOPLiArc0FilasTiempo345.csv')

# Agrupar por la columna 'Estacion' y sumar los valores de 'Entradas_E'
top_estaciones = filtered_time_data.groupby('Station')['Inputs'].sum()
top_estaciones
# Ordenar las estaciones por 'Entradas_E' en orden descendente y seleccionar las 10 primeras
estaciones_seleccionadas = ['(02000)Cabecera Autopista Norte', '(02300)Calle 100','(03000)Portal Suba', '(04000)Cabecera Calle 80','(05000)Portal Américas', '(06000)Portal Eldorado','(07000)Portal Sur','(08000)Portal Tunal','(09000)Cabecera Usme','(10000)Portal 20 de Julio']
#top_10_estaciones = top_estaciones.sort_values(ascending=False).head(10)
#top_10_estaciones

# Filtrar el dataset para incluir solo las 10 estaciones con la mayor cantidad de entradas
filtered_top_10_estaciones_data =filtered_time_data[filtered_time_data['Station'].isin(estaciones_seleccionadas)].copy()
filtered_top_10_estaciones_data

# Mostrar las primeras filas del nuevo DataFrame filtrado
#filtered_top_10_estaciones_data.to_csv('salidas20230809EngOPLiArc0FilasTiempo34510estaciones.csv')
filtered_top_10_estaciones_data
# Convirtiendo la columna 'Fecha_Transaccion' a tipo de dato de fecha
filtered_top_10_estaciones_data['Transaction_date'] = pd.to_datetime(filtered_top_10_estaciones_data['Transaction_date'])
# Crear la columna 'Dia_Semana'
filtered_top_10_estaciones_data['Day_Week_Number'] = filtered_top_10_estaciones_data['Transaction_date'].dt.dayofweek + 1

# Dias festivos

co_holidays = holidays.Colombia(years=[2023])
filtered_top_10_estaciones_data['Holidays'] = filtered_top_10_estaciones_data['Transaction_date'].apply(lambda x: x in co_holidays)
filtered_top_10_estaciones_data['Holidays'] = filtered_top_10_estaciones_data['Holidays'].astype(float)


filtered_top_10_estaciones_data['Transaction_date'] = pd.to_datetime(filtered_top_10_estaciones_data['Transaction_date'])
# Creando las nuevas columnas para año, mes y día
filtered_top_10_estaciones_data['Year'] = filtered_top_10_estaciones_data['Transaction_date'].dt.year
filtered_top_10_estaciones_data['Month'] = filtered_top_10_estaciones_data['Transaction_date'].dt.month
filtered_top_10_estaciones_data['Day'] = filtered_top_10_estaciones_data['Transaction_date'].dt.day

# Eliminando la columna original 'Fecha_Transaccion'
filtered_top_10_estaciones_data.drop('Transaction_date', axis=1,inplace=True)
filtered_top_10_estaciones_data

# Asegurándose de que 'Tiempo' sea un tipo de dato de hora
filtered_top_10_estaciones_data['Time'] = pd.to_datetime(filtered_top_10_estaciones_data['Time'], format='%H:%M:%S').dt.time

# Creando nuevas columnas para hora, minuto y segundo
filtered_top_10_estaciones_data['Hour'] = filtered_top_10_estaciones_data['Time'].apply(lambda x: x.hour)
filtered_top_10_estaciones_data['Minute'] = filtered_top_10_estaciones_data['Time'].apply(lambda x: x.minute)
filtered_top_10_estaciones_data['Second'] = filtered_top_10_estaciones_data['Time'].apply(lambda x: x.second)

# Eliminando la columna original 'Tiempo'
filtered_top_10_estaciones_data.drop(['Time','Second','Year'], axis=1,inplace=True)
# Mostrar las primeras filas del DataFrame modificado
filtered_top_10_estaciones_data

#ordered_columns = ['Month', 'Day', 'Hour', 'Minute', 'Station',
 #                  'Station_Access', 'Device', 'Inputs', 'Outputs']
ordered_columns = ['Month', 'Day', 'Day_Week_Number', 'Holidays', 'Hour', 'Minute', 'Station', 'Station_Access', 'Device', 'Inputs', 'Outputs']
data_reorganized = filtered_top_10_estaciones_data[ordered_columns].copy()

#data_reorganized.to_csv('construccion.csv')
data_reorganized['Inputs'].sum()
# Supongamos que quieres eliminar filas donde la columna 'ColumnaConCeros' tiene valores 0
data1_filtrado = data_reorganized.loc[data_reorganized['Inputs'] != 0]
#data_reorganized.info()
#data1_filtrado.info()

# Restablecer índices
data1_filtrado.reset_index(drop=True, inplace=True)

data1_filtrado.drop(['Station_Access','Device'],axis=1,inplace=True)
data1_filtrado




Ingrese la fecha de inicio (formato AAAA-MM-DD): 2024-01-01
Ingrese la fecha de fin (formato AAAA-MM-DD): 2024-01-31
Memory usage: 1357.3515625 megabytes
El script tardó 39.661930322647095 segundos en ejecutarse.


<ipython-input-1-f7a5a6709a89>:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1_filtrado.drop(['Station_Access','Device'],axis=1,inplace=True)


,Month,Day,Day_Week_Number,Holidays,Hour,Minute,Station,Inputs,Outputs
0,1,1,1,1.0,3,45,(07000)Portal Sur,1,0
1,1,1,1,1.0,3,45,(10000)Portal 20 de Julio,1,0
2,1,1,1,1.0,3,45,(04000)Cabecera Calle 80,1,0
3,1,1,1,1.0,3,45,(02000)Cabecera Autopista Norte,1,1
4,1,1,1,1.0,3,45,(02000)Cabecera Autopista Norte,1,0
...,...,...,...,...,...,...,...,...,...
572640,1,31,3,0.0,23,45,(04000)Cabecera Calle 80,1,3
572641,1,31,3,0.0,23,45,(08000)Portal Tunal,1,5
572642,1,31,3,0.0,23,45,(08000)Portal Tunal,2,4
572643,1,31,3,0.0,23,45,(08000)Portal Tunal,2,1


In [ ]:
data1_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572645 entries, 0 to 572644
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Month            572645 non-null  int64  
 1   Day              572645 non-null  int64  
 2   Day_Week_Number  572645 non-null  int64  
 3   Holidays         572645 non-null  float64
 4   Hour             572645 non-null  int64  
 5   Minute           572645 non-null  int64  
 6   Station          572645 non-null  object 
 7   Inputs           572645 non-null  int64  
 8   Outputs          572645 non-null  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 39.3+ MB


In [ ]:
data1_filtrado
# Crear el codificador OneHot
encoder = OneHotEncoder(sparse_output=False)

# Seleccionar las columnas a codificar
columns_to_encode = ['Station']

# Aplicar OneHot Encoding a las columnas seleccionadas
encoded_data = encoder.fit_transform(data1_filtrado[columns_to_encode])

# Crear un DataFrame con los datos codificados
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(columns_to_encode))

# Concatenar el DataFrame codificado con el original (excluyendo las columnas ya codificadas)
result = pd.concat([data1_filtrado.drop(columns_to_encode, axis=1), encoded_df], axis=1)
pd.set_option('display.max_columns', None)

# Mostrar las primeras filas del DataFrame resultante

#result.to_csv('validacionTroncal20230806to27EngTop10Cod.csv', index=False)
result.rename(columns={'Day_Number_Week': 'Day_Week_Number'}, inplace=True)
result.info()
result
#result.to_csv('datasetDiciembre4-10.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572645 entries, 0 to 572644
Data columns (total 17 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Month                                    572645 non-null  int64  
 1   Day                                      572645 non-null  int64  
 2   Day_Week_Number                          572645 non-null  int64  
 3   Holidays                                 572645 non-null  float64
 4   Hour                                     572645 non-null  int64  
 5   Minute                                   572645 non-null  int64  
 6   Inputs                                   572645 non-null  int64  
 7   Outputs                                  572645 non-null  int64  
 8   Station_(02000)Cabecera Autopista Norte  572645 non-null  float64
 9   Station_(03000)Portal Suba               572645 non-null  float64
 10  Station_(04000)Cabecera Calle 80

,Month,Day,Day_Week_Number,Holidays,Hour,Minute,Inputs,Outputs,Station_(02000)Cabecera Autopista Norte,Station_(03000)Portal Suba,Station_(04000)Cabecera Calle 80,Station_(05000)Portal Américas,Station_(06000)Portal Eldorado,Station_(07000)Portal Sur,Station_(08000)Portal Tunal,Station_(09000)Cabecera Usme,Station_(10000)Portal 20 de Julio
0,1,1,1,1.0,3,45,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1,1,1.0,3,45,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,1,1.0,3,45,1,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,1,1,1.0,3,45,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,1,1.0,3,45,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572640,1,31,3,0.0,23,45,1,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
572641,1,31,3,0.0,23,45,1,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
572642,1,31,3,0.0,23,45,2,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
572643,1,31,3,0.0,23,45,2,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
